In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
import utils.utils as uti
import process.file as f
import os
import tifffile
import cv2

In [2]:
datapath = os.path.join('/Users', 'danielsprague', 'FOCO_lab', 'data')

### Checking image statistics

In [44]:
def get_stats_FOCO(file):
    df = pd.DataFrame()

    imfile = sio.loadmat(file)

    data = imfile['data']
    channels = [0,1,2,3]
    RGBW = np.squeeze(data[:,:,:, channels])

    RGBW_flat = RGBW.reshape(-1, RGBW.shape[-1])

    min = np.amin(RGBW, axis = (0,1,2))
    max = np.amax(RGBW, axis = (0,1,2))
    mean = np.mean(RGBW, axis = (0,1,2))
    std = np.std(RGBW, axis = (0,1,2))
    
    print(std)

    return min, max, mean, std

In [45]:
get_stats_FOCO('data/NP_Ray/20220529-14-16-47/Composite.mat')

[ 21.10524233 114.68456053  21.50760003  71.32167479]


(array([72, 21, 30, 22], dtype=uint16),
 array([1738, 4095, 1646, 4095], dtype=uint16),
 array([174.04605165, 111.9973158 , 107.47043733, 104.03060069]),
 array([ 21.10524233, 114.68456053,  21.50760003,  71.32167479]))

### Histogram Matching and Median Filtering

In [125]:
def generate_histograms(image, scale):
    
    image = np.asarray(image)

    im_flat = image.reshape(-1, image.shape[-1])

    fig, axs = plt.subplots(2,2)

    for i, ax1 in enumerate(axs):
        for j, ax in enumerate(ax1):

            hist, bins = np.histogram(im_flat[:,i*2+j], 256, [0, 256*scale] )
            cdf = hist.cumsum()
            cdf_normalized = cdf * hist.max()/cdf.max()
            ax.axvline(np.max(im_flat[:,i*2+j]),linestyle='--') 
            ax.plot(cdf_normalized, color = 'b')
            ax.hist(im_flat[:,i*2+j], bins= np.arange(256)*scale, color= 'red')
            ax.set_xlabel('color channel gray count')
            ax.set_ylabel('pixel count')
            ax.set_xlim([0,256*scale])
            ax.legend(('max value', 'cdf', 'hist'), loc = 'upper right')
    
    axs[0,0].set_title('red histogram')
    axs[0,1].set_title('green histogram')
    axs[1,0].set_title('blue histogram')
    axs[1,1].set_title('white histogram')

    plt.show()


In [126]:
def equalize_hist(RGBW, threshs):
    '''
    thresh defines value above which to perform the histogram equalization
    loop through each pixel in image and transform based on histogram equalization
    '''

    size = RGBW.shape

    RGBW_new = np.zeros(size)

    flat = RGBW.reshape(-1, RGBW.shape[-1])

    for l in range(size[3]):
        channel = flat[:,l]

        thresh = threshs[l]
        
        hist_to_eq = channel[np.where(channel>=thresh)]
        N = len(hist_to_eq)
        num_bins = 4096-thresh
        hist, bins = np.histogram(hist_to_eq, num_bins, [thresh, 4096])
        cdf = hist.cumsum()
        
        for i in range(size[0]):
            for j in range(size[1]):
                for k in range(size[2]):
                        val = RGBW[i,j,k,l]

                        if val >= thresh:
                            val_index = np.where(bins==val)
                            cum_prob = cdf[val_index]/N
                            new_val = np.round(cum_prob*(num_bins-1))+thresh

                            RGBW_new[i,j,k,l] = new_val
                        
                        else:
                            RGBW_new[i,j,k,l] = val

    return RGBW_new

In [127]:
def match_histogram(A, ref, A_max, ref_max): 
    image = np.asarray(A)
    ref_im = np.asarray(ref)

    im_flat = image.reshape(-1, image.shape[-1]) #flatten images 
    ref_flat = ref_im.reshape(-1, ref_im.shape[-1])

    newim = np.zeros(A.shape)

    for l in range(image.shape[3]):
        chan_flat = im_flat[:,l]
        chan_ref_flat = ref_flat[:,l]

        hist, bins = np.histogram(chan_flat, A_max, [0, A_max]) #generate histograms
        refhist, refbins = np.histogram(chan_ref_flat, ref_max, [0,ref_max])

        cdf = hist.cumsum()/ chan_flat.size # generate cdf of histograms
        cdf_ref = refhist.cumsum()/ chan_ref_flat.size

        M = np.zeros(A_max) 

        for idx in range(A_max):
            ind = np.argmin(np.abs(cdf[idx]-cdf_ref)) # store pixel values with matching cdf from reference image
            M[idx] = ind

        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                for k in range(image.shape[2]):
                    newim[i,j,k,l] = M[A[i,j,k,l]]

    return newim

In [130]:
def median_filter(im, size):
    if size:
        size = size
    else:
        size=3

    for i in range(im.shape[2]):
        for j in range(im.shape[3]):
            im[:,:,i,j] = cv2.medianBlur(im[:,:,i,j], size)

    return im


In [138]:
folder = '20230322-22-43-03'
imfile = sio.loadmat(datapath +'/Manual_annotate/'+folder+'/composite.mat')

data = imfile['data']
channels = [0,1,2,3]
RGBW = np.squeeze(data[:,:,:,channels])

reffile = sio.loadmat(datapath +'/NP_paper/all/11_YAaLR.mat')
refchannels = reffile['prefs']['RGBW'][0][0]-1
refdata = reffile['data']
refRGBW = np.squeeze(refdata[:,:,:,refchannels])

newim = median_filter(RGBW, 3)
newim = match_histogram(newim, refRGBW, 4096, 65536)

sio.savemat(datapath+ '/Manual_annotate/'+folder+'/hist_med_image.mat', {'Hist_RGBW':newim})

imfile = sio.loadmat(datapath+ '/Manual_annotate/'+folder+'/hist_med_image.mat')
im = np.transpose(imfile['Hist_RGBW'],(2,3,0,1))
im = im.astype('uint16')
print(im.dtype)

tifffile.imwrite(datapath + '/Manual_annotate/'+folder+'/hist_med_image.tif', im, imagej = True)




uint16


In [132]:
newim = match_histogram(RGBW, refRGBW, 4096, 65536)

sio.savemat(datapath+ '/NP_Ray/'+folder+'/hist_match_image.mat', {'Hist_RGBW':newim})

imfile = sio.loadmat(datapath+ '/NP_Ray/'+folder+'/hist_match_image.mat')
im = np.transpose(imfile['Hist_RGBW'],(2,3,0,1))
im = im.astype('uint16')
print(im.dtype)

tifffile.imwrite(datapath + '/NP_Ray/'+folder+'/hist_match_image.tif', im, imagej = True)

uint16


In [141]:

reffile = sio.loadmat(datapath +'/NP_paper/all/11_YAaLR.mat')
refchannels = reffile['prefs']['RGBW'][0][0]-1
refdata = reffile['data']
refRGBW = np.squeeze(refdata[:,:,:,refchannels])

for folder in os.listdir(datapath+'/NP_FOCO_cropped'):
    if folder != ".DS_Store":
        imfile = sio.loadmat(datapath +'/NP_FOCO_cropped/'+folder+'/neuropal_1_MMStack_Pos0.ome.mat')

        data = imfile['data']
        channels = [0,2,4,1]
        RGBW = np.squeeze(data[:,:,:,channels])

        newim = median_filter(RGBW, 3)
        newim = match_histogram(newim, refRGBW, 4096, 65536)

        sio.savemat(datapath+ '/NP_FOCO_hist_med/'+folder+'/hist_med_image.mat', {'Hist_RGBW':newim})

        imfile = sio.loadmat(datapath+ '/NP_FOCO_hist_med/'+folder+'/hist_med_image.mat')
        im = np.transpose(imfile['Hist_RGBW'],(2,3,0,1))
        im = im.astype('uint16')
        print(im.dtype)

        tifffile.imwrite(datapath + '/NP_FOCO_hist_med/'+folder+'/hist_med_image.tif', im, imagej = True)

uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16
uint16


In [143]:
print(im.shape)

(45, 4, 240, 1000)


: 

### Convert to tiff

In [51]:
imfile = sio.loadmat('data/NP_Ray/20220529-14-16-47/'+'hist_equal_image.mat')
im = np.transpose(imfile['Hist_RGBW'],(2,3,0,1))
im = im.astype('uint16')
print(im.dtype)

tifffile.imwrite('data/NP_Ray/20220529-14-16-47/'+'hist_equal_image.ome.tif', im, imagej = True)

uint16


: 

### Update blobs.csv with new annotations

In [3]:
for folder in os.listdir(datapath+'/Manual_annotate'):
    if folder =='.DS_Store':
        continue
    for f in os.listdir(datapath+'/Manual_annotate/'+folder):
        if f == 'neuroPAL_image.csv':
            npal = pd.read_csv(datapath+'/Manual_annotate/'+folder+'/'+f, skiprows = 7)

        elif f == 'blobs.csv':
            blob = pd.read_csv(datapath+'/Manual_annotate/'+folder+'/'+f, index_col =0)
        elif f == 'neuroPAL_image.mat':
            imfile = sio.loadmat(datapath+'/Manual_annotate/'+folder+'/'+f)
            sz = imfile['data'].shape
            scale = np.asarray(imfile['info']['scale'][0][0]).flatten()

    npal= npal[['Real X (um)', 'Real Y (um)', 'Real Z (um)', 'User ID']]
    npal['Real X (um)'] = round(npal['Real X (um)']/scale[0])
    npal['Real Y (um)'] = round(npal['Real Y (um)']/scale[1])
    npal['Real Z (um)'] = round(npal['Real Z (um)']/scale[2])

    npal['Real X (um)'] = npal['Real X (um)'].astype('int32')
    npal['Real Y (um)'] = npal['Real Y (um)'].astype('int32')
    npal['Real Z (um)'] = npal['Real Z (um)'].astype('int32')

    revx = sz[1]+1- npal['Real X (um)']
    revy = sz[0]+1- npal['Real Y (um)']
    revz = sz[2]+1- npal['Real Z (um)']

    if np.abs(np.mean(np.asarray(npal['Real X (um)']))-np.mean(np.asarray(blob['X'])))>np.abs(np.mean(np.asarray(revx))-np.mean(np.asarray(blob['X']))):
        npal['Real X (um)'] = revx

    if np.abs(np.mean(np.asarray(npal['Real Y (um)']))-np.mean(np.asarray(blob['Y'])))>np.abs(np.mean(np.asarray(revy))-np.mean(np.asarray(blob['Y']))):
        npal['Real Y (um)'] = revy

    if np.abs(np.mean(np.asarray(npal['Real Z (um)']))-np.mean(np.asarray(blob['Z'])))>np.abs(np.mean(np.asarray(revz))-np.mean(np.asarray(blob['Z']))):
        npal['Real Z (um)'] = revz

    npal = npal.rename(columns={'Real X (um)':'X', 'Real Y (um)':'Y', 'Real Z (um)': 'Z'})
        
    blobs_new = pd.merge(blob, npal, how='right', on=['X', 'Y', 'Z'])

    blobs_new['status'] = blobs_new['status'].fillna(1.0)
    blobs_new['prov'] = blobs_new['prov'].fillna('curated')
    blobs_new = blobs_new.drop(columns=['ID'])
    blobs_new = blobs_new.rename(columns={'User ID':'ID'})
    blobs_new['blob'] = np.arange(len(blobs_new))

    blobs_new.to_csv(datapath+'/Manual_annotate/'+folder+'/blobs.csv')

In [120]:
folder = '20221106-21-23-19'
for f in os.listdir(datapath+'/Manual_annotate/'+folder):
    if f == 'neuroPAL_image.csv':
        npal = pd.read_csv(datapath+'/Manual_annotate/'+folder+'/'+f, skiprows = 7)

    elif f == 'blobs.csv':
        blob = pd.read_csv(datapath+'/Manual_annotate/'+folder+'/'+f, index_col =0)

    elif f == 'neuroPAL_image.mat':
        imfile = sio.loadmat(datapath+'/Manual_annotate/'+folder+'/'+f)
        sz = imfile['data'].shape
        scale = np.asarray(imfile['info']['scale'][0][0]).flatten()
        print(sz)

npal= npal[['Real X (um)', 'Real Y (um)', 'Real Z (um)', 'User ID']]
npal['Real X (um)'] = round(npal['Real X (um)']/scale[0])
npal['Real Y (um)'] = round(npal['Real Y (um)']/scale[1])
npal['Real Z (um)'] = round(npal['Real Z (um)']/scale[2])

npal['Real X (um)'] = npal['Real X (um)'].astype('int32')
npal['Real Y (um)'] = npal['Real Y (um)'].astype('int32')
npal['Real Z (um)'] = npal['Real Z (um)'].astype('int32')

revx = sz[1]+1- npal['Real X (um)']
revy = sz[0]+1- npal['Real Y (um)']
revz = sz[2]+1- npal['Real Z (um)']

if np.abs(np.mean(np.asarray(npal['Real X (um)']))-np.mean(np.asarray(blob['X'])))>np.abs(np.mean(np.asarray(revx))-np.mean(np.asarray(blob['X']))):
    npal['Real X (um)'] = revx
    print('X reverse')

if np.abs(np.mean(np.asarray(npal['Real Y (um)']))-np.mean(np.asarray(blob['Y'])))>np.abs(np.mean(np.asarray(revy))-np.mean(np.asarray(blob['Y']))):
    npal['Real Y (um)'] = revy
    print('Y reverse')

if np.abs(np.mean(np.asarray(npal['Real Z (um)']))-np.mean(np.asarray(blob['Z'])))>np.abs(np.mean(np.asarray(revz))-np.mean(np.asarray(blob['Z']))):
    npal['Real Z (um)'] = revz
    print('Z reverse')

print(npal.head())
print(blob.head())

(240, 1000, 48, 4)
Y reverse
Z reverse
   Real X (um)  Real Y (um)  Real Z (um) User ID
0          350          178           13     NaN
1          376          154           13     NaN
2          386          153           13     NaN
3          402          189           26     NaN
4          408          186           23     NaN
     blob    X    Y   Z  status  ID      prov
76     76  350  178  13       0 NaN  detected
68     68  376  154  13       0 NaN  detected
66     66  386  153  13       0 NaN  detected
183   183  402  189  26       0 NaN  detected
243   243  408  186  23       1 NaN   curated


In [116]:
print(sz)

(240, 1000, 48, 4)


In [11]:
data = sio.loadmat(datapath+'/Manual_annotate copy/20221106-21-47-31/neuroPAL_image.mat')

In [89]:
print(data['data'].shape)

(240, 1000, 48, 4)


In [96]:
scale = [0.3208, 0.3208, 0.75]
sz = [1000, 240, 48]

npal = pd.read_csv(datapath+'/Manual_annotate/20221106-21-47-31/neuroPAL_image.csv', skiprows=7)
npal= npal[['Real X (um)', 'Real Y (um)', 'Real Z (um)', 'User ID']]
blob = pd.read_csv(datapath+'/Manual_annotate/20221106-21-47-31/blobs.csv', index_col=0)
npal['Real X (um)'] = round(npal['Real X (um)']/scale[0])
npal['Real Y (um)'] = round(npal['Real Y (um)']/scale[1])
npal['Real Z (um)'] = round(npal['Real Z (um)']/scale[2])

npal['Real X (um)'] = npal['Real X (um)'].astype('int32')
npal['Real Y (um)'] = npal['Real Y (um)'].astype('int32')
npal['Real Z (um)'] = npal['Real Z (um)'].astype('int32')

revx = sz[0]+1- npal['Real X (um)']
revy = sz[1]+1- npal['Real Y (um)']
revz = sz[2]+1- npal['Real Z (um)']

if np.abs(np.mean(np.asarray(npal['Real X (um)']))-np.mean(np.asarray(blob['X'])))>np.abs(np.mean(np.asarray(revx))-np.mean(np.asarray(blob['X']))):
    npal['Real X (um)'] = revx

if np.abs(np.mean(np.asarray(npal['Real Y (um)']))-np.mean(np.asarray(blob['Y'])))>np.abs(np.mean(np.asarray(revy))-np.mean(np.asarray(blob['Y']))):
    npal['Real Y (um)'] = revy

if np.abs(np.mean(np.asarray(npal['Real Z (um)']))-np.mean(np.asarray(blob['Z'])))>np.abs(np.mean(np.asarray(revz))-np.mean(np.asarray(blob['Z']))):
    npal['Real Z (um)'] = revz

npal = npal.rename(columns={'Real X (um)':'X', 'Real Y (um)':'Y', 'Real Z (um)': 'Z'})
    
blobs = pd.merge(blob, npal, how='right', on=['X', 'Y', 'Z'])

blobs['status'] = blobs['status'].fillna(1.0)
blobs['prov'] = blobs['prov'].fillna('curated')
blobs = blobs.drop(columns=['ID'])
blobs = blobs.rename(columns={'User ID':'ID'})

blobs.to_csv(datapath+'/Manual_annotate/'+'20221106-21-47-31'+'/blobs_test.csv')

blobs.head()
#npal.head()


,blob,X,Y,Z,status,prov,ID
0,NaN,317,224,20,1.0,curated,NaN
1,81.0,344,120,32,0.0,detected,NaN
2,82.0,345,161,32,0.0,detected,NaN
3,77.0,364,154,31,0.0,detected,NaN
4,54.0,364,93,26,0.0,detected,I3


In [83]:
blobs.columns

Index(['blob', 'X', 'Y', 'Z', 'status', 'ID', 'prov', 'User ID'], dtype='object')

In [40]:
npal= npal[['Real X (um)', 'Real Y (um)', 'Real Z (um)', 'User ID']]
npal.head()

,Real X (um),Real Y (um),Real Z (um),User ID
0,101.694004,5.459252,21.75,NaN
1,110.355200,38.816800,12.75,NaN
2,110.676000,25.664000,12.75,NaN
3,116.771200,27.909600,13.50,NaN
4,116.771200,47.478400,17.25,I3


In [41]:
npal['Real X (um)'] = npal['Real X (um)']/0.3208
npal['Real Y (um)'] = npal['Real Y (um)']/0.3208
npal['Real Z (um)'] = npal['Real Z (um)']/0.75

npal.head()



,Real X (um),Real Y (um),Real Z (um),User ID
0,317.001259,17.017618,29.0,NaN
1,344.000000,121.000000,17.0,NaN
2,345.000000,80.000000,17.0,NaN
3,364.000000,87.000000,18.0,NaN
4,364.000000,148.000000,23.0,I3


In [33]:
blob.head()

,Unnamed: 0,blob,X,Y,Z,status,ID,prov
0,76,81,344,120,32,0,NaN,detected
1,77,82,345,161,32,0,NaN,detected
2,72,77,364,154,31,0,NaN,detected
3,53,54,364,93,26,0,NaN,detected
4,1,1,366,148,11,0,NaN,detected


In [34]:
npal['Real X (um)'] = npal['Real X (um)']
npal['Real Y (um)'] = 241 - npal['Real Y (um)']
npal['Real Z (um)'] = 49 - npal['Real Z (um)']

npal.head()

,Real X (um),Real Y (um),Real Z (um),User ID
0,317.001259,223.982382,20.0,NaN
1,344.000000,120.000000,32.0,NaN
2,345.000000,161.000000,32.0,NaN
3,364.000000,154.000000,31.0,NaN
4,364.000000,93.000000,26.0,I3


In [35]:
blob.head()

,Unnamed: 0,blob,X,Y,Z,status,ID,prov
0,76,81,344,120,32,0,NaN,detected
1,77,82,345,161,32,0,NaN,detected
2,72,77,364,154,31,0,NaN,detected
3,53,54,364,93,26,0,NaN,detected
4,1,1,366,148,11,0,NaN,detected


In [38]:
print(np.mean(np.asarray(npal['Real X (um)'])))
print(np.mean(np.asarray(blob['X'])))
print(np.mean(np.asarray(npal['Real Y (um)'])))
print(np.mean(np.asarray(blob['Y'])))
print(np.mean(np.asarray(npal['Real Z (um)'])))
print(np.mean(np.asarray(blob['Z'])))

475.2822106640726
475.45217391304345
113.45566087475275
113.02608695652174
26.698275862068964
26.73913043478261
